In [ ]:
# Installation des dépendances nécessaires
!pip install -q langchain-google-genai pypdf langchain-chroma faiss-cpu sentence-transformers streamlit
!pip install langchain_community langchainhub chromadb langchain
!pip install -q python-dotenv # Pour gérer les variables d'environnement (API key)
!pip install -q transformers # Pour le modèle d'embedding HuggingFace

# Assurons-nous que LangChain est à jour pour les dernières fonctionnalités
!pip install -q --upgrade langchain

print("Toutes les dépendances ont été installées ou mises à jour.")

from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
userdata.get('LANGCHAIN_API_KEY')


import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

Toutes les dépendances ont été installées ou mises à jour.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Path to the pdf file
pdf_path = '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf' # chemin d'accès au document qui constitura notre corpus
# pdf_path = "/content/drive/MyDrive/Colab Notebooks/AI31/lo17_rag/data/data_50_page.pdf"
# !ls "/content/drive/MyDrive/Colab Notebooks/AI31/lo17_rag/data/data_50_page.pdf"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Objectif du projet
L'objectif de ce projet est celui de la mise en place d'un système RAG (Retrieval Augmented Generation) dédié à la partie législative du Code du travail (plus précisement jusqu'à la Partie II - Livre II : La négociation collective - Les conventions et accords collectifs de travail, Sous-section 3, paragraphe 1er - Egalité professionnelle entre les femmes et les hommes), servant d’**outil de requêtage intelligent** et d’**aide à la décision** pour des boîtes de **consultation juridique**.

L’objectif est de permettre aux utilisateurs (juristes, consultants, avocats juniors, etc.) de poser des questions en langage naturel et d’obtenir des réponses précises, contextualisées et appuyées par des sources juridiques fiables (textes de loi, jurisprudence, doctrine).

**NB :** le RAG qui sera mis sur pied sera spécifique a un sous ensemble de la partie législative du code droit du travail. Ce serait très ambitieux de notre part de mettre sur pied un RAG couvrant l’ensemble du droit français étant donné le temps imparti pour la réalisation de ce projet (le droit étant vaste, technique et complexe).

# ETAPES CLES
1. **Préparation de l'environnement :** Installer les outils et librairies nécessaires.
2. **Chargement et découpage du document (Document Loading & Splitting) :**
Chargement du pdf de 278 pages (code du travail). Un document aussi volumineux ne peut pas être traité d'un seul bloc par un modèle de langage. Nous devrons le découper en morceaux plus petits (appelés "chunks" ou "morceaux") tout en conservant le contexte. C'est crucial pour la pertinence des recherches.
3. **Création des embeddings (Embeddings Generation) :**
Chaque morceau de texte sera transformé en une représentation numérique appelée "embedding". C'est une sorte de "vecteur" qui capture la signification sémantique du texte. Ces embeddings nous permettront de trouver des morceaux de texte similaires à la question poser par un utilisateur (sous forme de requête).
3. **Stockage vectoriel avec Chroma (Vector Store with ChromaDB) :**
Nous allons stocker ces embeddings, ainsi que le texte original des morceaux, dans une base de données vectorielle appelée Chroma.
Chroma est optimisé pour les recherches de similarité vectorielle, ce qui est exactement ce dont nous avons besoin pour récupérer les informations pertinentes.
4. **Construction de la chaîne RAG avec LangChain (Building the RAG Chain with LangChain) :**
LangChain est un framework qui aide à orchestrer les différentes étapes (récupération de documents, génération de réponses).
Nous utiliserons LangChain pour relier notre base de données vectorielle (Chroma) et notre modèle de langage (Gemini).
5. **Intégration du modèle de langage Gemini (LLM Integration with Gemini) :**
Gemini sera le cerveau de notre application. Il prendra les morceaux de texte pertinents et la question pour générer une réponse cohérente et informative.
6. **Construction de l'interface utilisateur avec Streamlit (Streamlit UI) :**
Streamlit nous permettra de créer une interface web simple et interactive où l'utilisateur pourras poser ses questions et voir les réponses, y compris les sources.
7. **Gestion des sources/Gestion des Hallucinations (Source Retrieval) :**
Nous nous assurerons que chaque réponse est accompagnée de l'indication de la page ou du morceau de texte d'où provient l'information.

## Pourquoi ces outils ?

1. **LangChain :** C'est un framework puissant qui simplifie énormément la construction d'applications basées sur les grands modèles de langage (LLMs). Il offre des composants modulaires pour le chargement de documents, le découpage, les bases de données vectorielles, et l'intégration de LLMs.
2. **Chroma :** Une base de données vectorielle légère et facile à utiliser, parfaite pour les projets de taille moyenne et l'apprentissage. Elle est souvent utilisée avec LangChain.
3. **Gemini :** Un modèle de langage performant de Google, capable de comprendre le langage naturel et de générer des réponses pertinentes.
4. **Streamlit :** C'est un excellent outil pour construire rapidement des interfaces utilisateur pour des applications de machine learning et de data science, sans avoir besoin de connaissances approfondies en développement web.

## PARTIE INDEXATION


In [ ]:
import re
from typing import List, Optional

from langchain.text_splitter import TextSplitter
from langchain.schema import Document

class TitleBasedSplitter(TextSplitter):
    """
    Splitter de texte basé sur la détection de titres.
    Garde les métadonnées de la page d'origine pour chaque chunk.
    """
    def __init__(self, pattern: str = r"(Titre\s+(?:[IVXLCDM]+(?:er|ème)?|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:)"):
        """
        Initialise le splitter avec un pattern regex pour les titres.
        Le pattern par défaut cherche "Titre " suivi de:
        - Soit des chiffres romains (I, V, X...) optionnellement suivis de "er" ou "ème".
        - Soit des mots (lettres, chiffres, accents, apostrophes, tirets).
        - Le tout suivi d'un deux-points (avec espaces optionnels autour).
        Exemples: "Titre Ier :", "Titre préliminaire :", "Titre II :"
        """
        super().__init__()
        self.compiled_pattern = re.compile(pattern, re.IGNORECASE)

    def split_documents(self, documents: List[Document], **kwargs) -> List[Document]:
        """
        Divise une liste de documents (pages) en chunks basés sur les titres.
        Chaque chunk hérite des métadonnées de son document d'origine.
        """
        all_chunks: List[Document] = []
        for doc in documents:
            text = doc.page_content
            metadata = doc.metadata.copy() # Copie des métadonnées de la page d'origine

            matches = list(self.compiled_pattern.finditer(text))

            if not matches:
                # Si aucun titre n'est trouvé sur la page, la page entière est un chunk
                if text.strip(): # S'assurer que le texte n'est pas vide
                    all_chunks.append(Document(page_content=text.strip(), metadata=metadata))
                continue # Passer à la page suivante

            # Cas où il y a du texte avant le premier titre sur la page
            if matches[0].start() > 0:
                pre_title_text = text[0:matches[0].start()].strip()
                if pre_title_text:
                    all_chunks.append(Document(page_content=pre_title_text, metadata=metadata))

            for i in range(len(matches)):
                start = matches[i].start()
                # La fin du chunk est le début du prochain match, ou la fin du texte si c'est le dernier match
                end = matches[i+1].start() if i + 1 < len(matches) else len(text)

                chunk_content = text[start:end].strip()

                if chunk_content:
                    # Ajout de métadonnées spécifiques au chunk si besoin,
                    # par exemple le titre identifié ou une ID de chunk
                    chunk_metadata = metadata.copy()
                    # On peut ajouter ici des infos comme le titre spécifique du chunk
                    # chunk_metadata["title"] = matches[i].group(0).strip()
                    all_chunks.append(Document(page_content=chunk_content, metadata=chunk_metadata))

        return all_chunks

    # Nous n'avons pas besoin de split_text si nous utilisons split_documents directement sur des objets Document.
    # Cependant, si l'on veux conserver la compatibilité avec d'autres TextSplitter,
    # l'on peut implémenter split_text pour une chaîne unique.
    def split_text(self, text: str) -> List[str]:
        """
        Divise une chaîne de texte en chunks basés sur les titres.
        Cette méthode est appelée par split_documents si un Document est passé.
        """
        chunks = []
        matches = list(self.compiled_pattern.finditer(text))

        if not matches:
            return [text.strip()] if text.strip() else []

        # Ajouter le texte avant le premier titre comme un chunk si non vide
        if matches[0].start() > 0:
            pre_title_text = text[0:matches[0].start()].strip()
            if pre_title_text:
                chunks.append(pre_title_text)

        for i in range(len(matches)):
            start = matches[i].start()
            end = matches[i+1].start() if i + 1 < len(matches) else len(text)
            chunk_content = text[start:end].strip()
            if chunk_content:
                chunks.append(chunk_content)
        return chunks

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document


if not os.path.exists(pdf_path):
    print(f"ERREUR: Le fichier PDF '{pdf_path}' n'a pas été trouvé. Veuillez vérifier le chemin.")
else:
    print(f"Chargement du PDF depuis : {pdf_path}")

    # 1. On charge les documents page par page
    loader = PyPDFLoader(pdf_path)
    docs_from_loader = loader.load() # Chaque élément est un Document (page) avec ses métadonnées

    if not docs_from_loader:
        print("Aucun document n'a pu être chargé par PyPDFLoader. Veuillez vérifier le PDF.")
        documents = [] # L'on s'assure que 'documents' est défini même si le chargement échoue
    else:
        print(f"Nombre de pages chargées par PyPDFLoader : {len(docs_from_loader)}")

        # 2. Initialiser notre splitter basé sur les titres
        splitter = TitleBasedSplitter()

        # 3. Appliquer le splitter sur chaque document (page)
        # La méthode split_documents prend une liste de Document et retourne une liste de Document.
        documents = splitter.split_documents(docs_from_loader)

        print(f"Nombre de documents (chunks) après TitleBasedSplitter : {len(documents)}")

        # Affichage de quelques exemples de chunks pour vérifier leur contenu et leurs métadonnées
        for i, doc_chunk in enumerate(documents[:5]): # Affiche les 5 premiers chunks
            print(f"\n--- Chunk {i+1} ---")
            print(f"Page content (début): {doc_chunk.page_content[:500]}...") # Affiche les 500 premiers caractères
            print(f"Metadata: {doc_chunk.metadata}")

            # Vérifier si 'page' est dans les métadonnées
            if 'page' in doc_chunk.metadata:
                print(f"Source Page: {doc_chunk.metadata['page'] + 1}") # +1 car les pages sont souvent 0-indexées

# NB : Il est important de s'assurer qu'il n'y a pas d'erreur dans 'documents' si aucun document n'est chargé.
if not documents:
    print("Le traitement ne peut pas continuer car aucun document n'a été créé.")
    sys.exit()

Chargement du PDF depuis : /content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf
Nombre de pages chargées par PyPDFLoader : 278
Nombre de documents (chunks) après TitleBasedSplitter : 312

--- Chunk 1 ---
Page content (début): Partie législative - Chapitre préliminaire : Dialogue social. 
Partie législative
Chapitre préliminaire : Dialogue social.
L. 1  LOI n°2008-67 du 21 janvier 2008 - art. 3      
  Legif.   
  Plan   
  Jp.Judi.   
  Jp.Admin.   
  Juricaf  
Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives
du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et
interprofessionnelle fait l'objet d'une conc...
Metadata: {'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-06-03T20:04:09+00:00', 'source': '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf', 'total_pages': 278, 'page': 0, 'page_label': '1'}
Source Page: 1

--- C

In [ ]:
import re
from typing import List, Optional, Dict, Any

from langchain.text_splitter import TextSplitter
from langchain.schema import Document

class CodeDuTravailStructureExtractor(TextSplitter):
    """
    Splitter et extracteur de structure pour le Code du Travail.
    Découpe le document en chunks et enrichit les métadonnées
    avec les informations de Partie, Livre, Titre, Chapitre, Section et Article.
    """
    def __init__(
        self,
        # Pattern pour les titres principaux (Titre Ier :, Titre Préliminaire :)
        title_pattern: str = r"(Titre\s+(?:[IVXLCDM]+(?:er|ème)?|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
        # Pattern pour les chapitres (Chapitre Ier :, Chapitre Unique :)
        chapter_pattern: str = r"(Chapitre\s+(?:[IVXLCDM]+(?:er|ème)?|unique|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
        # Pattern pour les sections (Section 1 :, Section unique :)
        section_pattern: str = r"(Section\s+(?:\d+|unique|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
        # Pattern pour les articles (L. 123-1, R. 456-7, D. 789-10)
        article_pattern: str = r"(Article\s+((?:L|R|D)\.\s*\d{3}-\d+(?:-\d+)?(?:-\d+)?)[\s\S]*?(?=Article\s+((?:L|R|D)\.\s*\d{3}-\d+(?:-\d+)?(?:-\d+)?)|Titre\s+|Chapitre\s+|Section\s+|$))",
        # Capture le numéro de l'article dans un groupe pour extraction facile
        article_num_capture_group: int = 2, # Le groupe qui contient "L. 123-1"
        keep_separator: bool = True, # Indique si le séparateur (titre/chapitre/article) doit faire partie du chunk
        **kwargs
    ):
        super().__init__(keep_separator=keep_separator, **kwargs)
        self.title_pattern = re.compile(title_pattern, re.IGNORECASE | re.DOTALL)
        self.chapter_pattern = re.compile(chapter_pattern, re.IGNORECASE | re.DOTALL)
        self.section_pattern = re.compile(section_pattern, re.IGNORECASE | re.DOTALL)
        self.article_pattern = re.compile(article_pattern, re.IGNORECASE | re.DOTALL)
        self.article_num_capture_group = article_num_capture_group

    def split_documents(self, documents: List[Document]) -> List[Document]:
        """
        Divise les documents (pages) en chunks et enrichit les métadonnées.
        """
        all_chunks: List[Document] = []
        current_book = None
        current_part = None
        current_title = None
        current_chapter = None
        current_section = None

        # Pour le Code du Travail, la "Partie législative" et "Partie réglementaire"
        # sont souvent des sections de haut niveau. On pourrait les détecter aussi.
        # Par exemple: r"(Partie\s+(?:législative|réglementaire)\s*:\s*.+?)(?=\n(?:Partie\s|Livre\s|Titre\s|Chapitre\s|Section\s|Article\s|$))"

        for doc in documents:
            text = doc.page_content
            page_metadata = doc.metadata.copy() # Copier les métadonnées de la page (ex: page number)

            # Il est crucial de détecter les éléments hiérarchiques dans l'ordre décroissant
            # (Partie -> Livre -> Titre -> Chapitre -> Section -> Article)
            # pour s'assurer que les métadonnées sont correctement propagées.

            # Nous allons traiter le texte de la page pour identifier les blocs.
            # L'idée est de trouver tous les matchs de titres, chapitres, sections, articles
            # et de les traiter séquentiellement.

            # Simple approche pour commencer: On va splitter principalement par article
            # et ensuite extraire les informations de titre/chapitre/section
            # qui précèdent cet article ou sont détectées dans le chunk de l'article.

            # Cette méthode nécessite une logique plus complexe pour propager le contexte
            # (quel titre/chapitre actuel s'applique à un article).
            # On va utiliser une approche par "contexte courant" qui est mise à jour à chaque détection.

            # Initialisation du contexte pour cette page (on garde le dernier contexte connu)
            # Pour la première page, ces valeurs seront None.
            # Pour les pages suivantes, elles hériteront des valeurs de la fin de la page précédente.


            # Pour l'instant, on suppose que les titres sont répétés ou que les chunks sont suffisamment petits
            # pour qu'un article soit dans son contexte de titre/chapitre.
            # C'est un point clé à affiner si les sources ne sont pas précises.

            # Découpage par Article principalement
            article_matches = list(self.article_pattern.finditer(text))

            last_idx = 0
            # Si du texte précède le premier article
            if article_matches and article_matches[0].start() > 0:
                pre_article_text = text[0:article_matches[0].start()].strip()
                if pre_article_text:
                    # Tentative d'extraction des infos de titre/chapitre/section pour ce bloc introductif
                    # C'est une simplification, idéalement on détecterait ces éléments en amont du split.
                    chunk_metadata = self._extract_hierarchy_metadata(pre_article_text, page_metadata)
                    all_chunks.append(Document(page_content=pre_article_text, metadata=chunk_metadata))
                last_idx = article_matches[0].start()

            for i, match in enumerate(article_matches):
                article_content = match.group(0).strip() # Le contenu complet de l'article (incluant le numéro)
                article_number = match.group(self.article_num_capture_group) # Le numéro de l'article

                # Ici, nous pourrions affiner l'extraction de la date ou de la loi spécifique
                # si un pattern est détectable dans l'article_content
                # Par exemple: LOI n° 2014-288 du 5 mars 2014-art. 29 (V)
                # pattern_date_loi = r"(LOI n°\s*\d{4}-\d+\s*du\s+\d{1,2}\s+\w+\s+\d{4})"
                # date_loi_match = re.search(pattern_date_loi, article_content)
                # if date_loi_match:
                #     article_metadata["date_loi"] = date_loi_match.group(1)

                chunk_metadata = page_metadata.copy()
                chunk_metadata["type"] = "Article"
                chunk_metadata["article_number"] = article_number

                # Tenter d'extraire le contexte hiérarchique pour cet article
                # On recherche les titres, chapitres, sections qui précèdent immédiatement cet article
                # dans la portion de texte traitée pour cette page.

                # C'est une simplification pour l'exemple.
                # Une vraie solution robuste impliquerait de parser le document séquentiellement
                # et de maintenir un état des "titres actifs" et "chapitres actifs"
                # au fur et à mesure que l'on lit le document.

                # Pour l'instant, on fait une extraction locale:
                chunk_metadata.update(self._extract_hierarchy_metadata(article_content, page_metadata))

                all_chunks.append(Document(page_content=article_content, metadata=chunk_metadata))
                last_idx = match.end()

            # Si du texte reste après le dernier article sur la page
            if last_idx < len(text):
                remaining_text = text[last_idx:].strip()
                if remaining_text:
                    chunk_metadata = self._extract_hierarchy_metadata(remaining_text, page_metadata)
                    all_chunks.append(Document(page_content=remaining_text, metadata=chunk_metadata))

        return all_chunks

    def _extract_hierarchy_metadata(self, text: str, base_metadata: Dict[str, Any]) -> Dict[str, Any]:
        """
        Tente d'extraire les informations de hiérarchie (Titre, Chapitre, Section)
        à partir d'un morceau de texte et les ajoute aux métadonnées.
        """
        metadata = base_metadata.copy()

        # Chercher le titre le plus proche
        title_match = self.title_pattern.search(text)
        if title_match:
            metadata["title"] = title_match.group(1).strip()

        # Chercher le chapitre le plus proche
        chapter_match = self.chapter_pattern.search(text)
        if chapter_match:
            metadata["chapter"] = chapter_match.group(1).strip()

        # Chercher la section la plus proche
        section_match = self.section_pattern.search(text)
        if section_match:
            metadata["section"] = section_match.group(1).strip()


        # Pour les livres et parties, c'est plus complexe car ils sont souvent en début de document
        # ou indiqués par des en-têtes/pieds de page non extraits par `page_content`.
        # Pour le "Livre" et la "Partie", il est souvent plus efficace de les extraire
        # soit manuellement au début, soit en faisant une passe initiale sur le document complet
        # pour établir la hiérarchie.
        # Pour l'instant, je ne les inclus pas dans _extract_hierarchy_metadata,
        # car un chunk d'article ne les contiendra pas nécessairement.
        # On pourrait les ajouter comme des métadonnées globales au document si elles sont fixes,
        # ou les propager depuis un parseur de document entier.

        return metadata

    # La méthode split_text n'est pas utilisée directement par split_documents,
    # mais on la garde pour la conformité avec la classe mère, même si elle n'est pas optimisée
    # pour notre usage actuel d'extraction de métadonnées.
    def split_text(self, text: str) -> List[str]:
        # Cette implémentation est plus simple, mais ne gère pas la propagation des métadonnées
        # C'est pourquoi nous utiliserons principalement split_documents.
        chunks = []
        # On va splitter par les articles pour avoir des chunks plus petits
        article_splits = self.article_pattern.split(text)

        # Si le premier élément n'est pas un séparateur, c'est un préambule
        if len(article_splits) > 0 and not self.article_pattern.match(article_splits[0]):
            if article_splits[0].strip():
                chunks.append(article_splits[0].strip())
            start_index = 1
        else:
            start_index = 0

        # Réassembler les articles avec leur numéro
        for i in range(start_index, len(article_splits), self.article_num_capture_group + 1):
            if i + self.article_num_capture_group < len(article_splits):
                article_num = article_splits[i + self.article_num_capture_group -1] # Le numéro de l'article capturé
                article_content = article_splits[i + self.article_num_capture_group] # Le contenu après le numéro

                full_article_chunk = f"Article {article_num.strip()} {article_content.strip()}"
                if full_article_chunk.strip():
                    chunks.append(full_article_chunk.strip())
            elif article_splits[i].strip():
                chunks.append(article_splits[i].strip())
        return chunks

In [ ]:
import sys

if not os.path.exists(pdf_path):
    print(f"ERREUR: Le fichier PDF '{pdf_path}' n'a pas été trouvé. Veuillez vérifier le chemin.")
    documents = [] # On s'assure que 'documents' est défini même si le fichier est manquant
else:
    print(f"Chargement du PDF depuis : {pdf_path}")

    loader = PyPDFLoader(pdf_path)
    # Chargement page par page pour conserver les métadonnées de page
    docs_from_loader = loader.load()

    if not docs_from_loader:
        print("Aucun document n'a pu être chargé par PyPDFLoader. Veuillez vérifier le PDF.")
        documents = []
    else:
        print(f"Nombre de pages chargées par PyPDFLoader : {len(docs_from_loader)}")

        # Initialisation de l'extracteur de structure
        # On va créer des patterns plus robustes si le test initial ne donne pas satisfaction
        # Les patterns par défaut dans la classe sont un bon point de départ.
        structure_extractor = CodeDuTravailStructureExtractor()

        # Appliquer l'extracteur sur les documents (pages)
        documents = structure_extractor.split_documents(docs_from_loader)

        print(f"Nombre de documents (chunks enrichis) après extraction de structure : {len(documents)}")

        # Afficher quelques exemples de chunks enrichis
        for i, doc_chunk in enumerate(documents[:5]): # Affiche les 5 premiers chunks
            print(f"\n--- Chunk {i+1} ---")
            print(f"Page content (début): {doc_chunk.page_content[:500]}...")
            print(f"Metadata: {doc_chunk.metadata}")
            # Exemple de vérification des métadonnées spécifiques
            if 'page' in doc_chunk.metadata:
                print(f"  Source Page: {doc_chunk.metadata['page'] + 1}")
            if 'type' in doc_chunk.metadata:
                print(f"  Type de chunk: {doc_chunk.metadata['type']}")
            if 'article_number' in doc_chunk.metadata:
                print(f"  Numéro d'Article: {doc_chunk.metadata['article_number']}")
            if 'title' in doc_chunk.metadata:
                print(f"  Titre: {doc_chunk.metadata['title']}")
            if 'chapter' in doc_chunk.metadata:
                print(f"  Chapitre: {doc_chunk.metadata['chapter']}")

# NB : S'assurer que 'documents' est défini pour les étapes suivantes même en cas d'échec de chargement
if not documents:
    print("Le traitement ne peut pas continuer car aucun document structuré n'a été créé.")
    sys.exit()

Chargement du PDF depuis : /content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf
Nombre de pages chargées par PyPDFLoader : 278
Nombre de documents (chunks enrichis) après extraction de structure : 411

--- Chunk 1 ---
Page content (début): Partie législative - Chapitre préliminaire : Dialogue social. 
Partie législative
Chapitre préliminaire : Dialogue social.
L. 1  LOI n°2008-67 du 21 janvier 2008 - art. 3      
  Legif.   
  Plan   
  Jp.Judi.   
  Jp.Admin.   
  Juricaf  
Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives
du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et
interprofessionnelle fait l'objet d'une conc...
Metadata: {'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-06-03T20:04:09+00:00', 'source': '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf', 'total_pages': 278, 'page': 0, 'page_label': '1', 'chapte

## PARTIES EMBEDDING, STOCKAGE VECTORIELLE, CHAINE RAG, LLM

### Pourquoi découper le document (Chunking) ?

- **Limitation des LLM :** Les grands modèles de langage ont une "fenêtre de contexte" limitée. C'est la quantité de texte qu'ils peuvent lire et traiter en une seule fois. Un PDF de 2755 pages est bien au-delà de ce qu'ils peuvent gérer.
- **Pertinence de la recherche :** Si l'on cherche un concept dans 2775 pages, la recherche sera très lente et peu précise.
*  _Solution :_
  - **Le découpage (Chunking)**
     - Nous allons diviser le PDF en petits morceaux (des paragraphes, quelques phrases, etc.). Chaque morceau est suffisamment petit pour être traité par un LLM et suffisamment grand pour conserver du sens.
*  _Conséquence :_ Quand un utilisateur posera une question, les morceaux les plus pertinenets seront chercher et non l'intégralité du document.


### Qu'est-ce qu'un Embedding et pourquoi en a-t-on besoin ?
Imaginons que chaque mot, phrase ou morceau de texte peut être représenté par un point dans un espace multidimensionnel (un peu comme des coordonnées X, Y, Z, mais avec beaucoup plus de dimensions).

- **Transformation en nombres :** Un "embedding" est une suite de nombres (un vecteur) qui représente la signification sémantique d'un texte. Des textes qui ont un sens similaire seront représentés par des vecteurs "proches" dans cet espace.
- **Recherche de similarité :** La question de l'utilisateur sera transformée en embedding. Ensuite, une recherche des embeddings de morceaux de texte qui sont les plus "proches" (on pourra notamment faire usage de l'algorithme des K-plus proches voisins pour récupérer les informations les plus pertinentes) de l'embedding da la question se fera dans la base de données. C'est la base de la recherche sémantique.

* _Pourquoi ?_
  - Un ordinateur ne "comprend" pas le texte. Il comprend les nombres. Les embeddings sont le pont entre le langage humain et la capacité de l'ordinateur à trouver des similarités de sens.

### Le rôle de la base de données vectorielle (ChromaDB)
Une fois que nous avons nos morceaux de texte et leurs embeddings, où les stockons-nous de manière efficace pour pouvoir les rechercher rapidement ?

- **Base de données traditionnelle vs. Base de données vectorielle :** Une base de données classique (comme SQL) est optimisée pour des recherches exactes (ex: "trouve tous les utilisateurs dont le nom est 'Dupont'"). Une base de données vectorielle est optimisée pour des recherches de similarité (ex: "trouve tous les textes qui sont similaires à 'licenciement abusif'").

**ChromaDB :** C'est une base de données vectorielle légère et facile à mettre en place. Elle va stocker :
- Les morceaux de texte bruts (le contenu original).
- Les embeddings correspondants (les vecteurs numériques).
- Des métadonnées (comme le numéro de page d'où vient le morceau, ce qui est crucial pour les sources !).
_Fonctionnement :_ Quand l'utilisateur pose une question, Chroma va rapidement identifier les embeddings les plus similaires à sa question, et lui retourner les morceaux de texte correspondants.


### LangChain : L'orchestrateur de notre application
LangChain est un framework puissant qui simplifie énormément la construction d'applications basées sur les grands modèles de langage.

_Concept de "Chains" (Chaînes) :_ LangChain permet de relier différents composants (comme le chargement de documents, les découpeurs, les modèles d'embeddings, les bases de données vectorielles, et les LLM) dans une séquence logique, une "chaîne".

- **Modularité :** Chaque partie de l'application RAG (chargement, découpage, embedding, recherche, génération) est un "maillon" de la chaîne. LangChain permet de les assembler facilement.
- **Simplification :** Au lieu de gérer manuellement chaque interaction entre ces composants, LangChain fournit des abstractions qui rendent le développement plus rapide et plus propre.


### Gemini : Le cerveau qui génère la réponse
Gemini est le modèle de langage (LLM) de Google que nous allons utiliser.

_Son rôle :_ Une fois que Chroma a récupéré les morceaux de texte les plus pertinents du document/PDF, Gemini va recevoir :
- La question.
- Les morceaux de texte pertinents.

_Sa tâche :_ Il va analyser ces informations et générer une réponse cohérente, synthétisée, et basée sur le contexte fourni. C'est là que la "Génération" de RAG entre en jeu.
- **API :** Nous interagirons avec Gemini via son API (Interface de Programmation d'Application), ce qui signifie que nous enverrons des requêtes à un service Google pour obtenir des réponses.

### Streamlit : L'interface utilisateur simple et rapide
Pour interagir avec notre application, nous avons besoin d'une interface.

####Qu'est-ce que c'est ?
**Streamlit** est une bibliothèque Python qui permet de créer très facilement des applications web interactives pour la science des données et le machine learning.
*  _Avantages :_ Pas besoin d'être un expert en développement web (HTML, CSS, JavaScript). Avec quelques lignes de Python, tu peux créer des widgets (champs de texte, boutons) et afficher des résultats.

_Notre utilisation :_ Nous l'utiliserons pour créer une boîte de texte où l'utilisateur pourras taper ses questions, un bouton pour les soumettre, et un espace pour afficher la réponse de Gemini et les sources.

In [ ]:
!pip install -q rank_bm25

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever


# S'assurer que 'documents' n'est pas vide avant de continuer
if not documents:
    print("ATTENTION : La liste 'documents' est vide. Le processus RAG ne peut pas continuer.")
    exit("Processus arrêté car aucun document n'a été traité.")

print("\n--- Étape 4 : Création des Embeddings ---")
embedding_model_name = "all-MiniLM-L6-v2"
print(f"Initialisation du modèle d'embedding : {embedding_model_name}")

try:
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    print("Modèle d'embedding initialisé avec succès.")
except Exception as e:
    print(f"ERREUR lors de l'initialisation du modèle d'embedding : {e}")
    print("Veuillez vérifier votre connexion internet et les dépendances 'sentence-transformers'.")
    exit("Processus arrêté car le modèle d'embedding n'a pas pu être initialisé.")


print("\n--- Étape 5 : Création ou chargement du Vectorstore Chroma ---")
chroma_db_dir = "./chroma_db_codetravail"

if os.path.exists(chroma_db_dir) and os.listdir(chroma_db_dir):
    print(f"Chargement du vectorstore Chroma existant depuis : {chroma_db_dir}")
    try:
        vectorstore = Chroma(persist_directory=chroma_db_dir, embedding_function=embedding_model)
        print("Vectorstore Chroma chargé.")
    except Exception as e:
        print(f"ERREUR lors du chargement du vectorstore Chroma : {e}")
        print("Tentative de recréation du vectorstore...")
        vectorstore = Chroma.from_documents(documents, embedding=embedding_model, persist_directory=chroma_db_dir)
        print("Vectorstore Chroma recréé.")
else:
    print(f"Création d'un nouveau vectorstore Chroma dans : {chroma_db_dir}")
    vectorstore = Chroma.from_documents(documents, embedding=embedding_model, persist_directory=chroma_db_dir)
    print("Vectorstore Chroma créé.")


# --- Configuration des Retrievers Hybrides ---
print("\n--- Étape 5.5 : Configuration des Retrievers Hybrides ---")

# 1. Retriever sémantique (Chroma Retriever)
semantic_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 2. Retriever basé sur les mots-clés (BM25)
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 5 # Nombre de documents à récupérer pour BM25

# 3. Combinaison des retrievers avec EnsembleRetriever
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, semantic_retriever],
    weights=[0.5, 0.5] # Poids égaux pour commencer
)
print("Retrievers hybrides (BM25 et Sémantique) configurés avec succès.")
print("La recherche utilisera la fusion des rangs.")

# LE RETRIEVER PRINCIPAL UTILISÉ DANS LA CHAÎNE EST MAINTENANT LE HYBRIDE
retriever = hybrid_retriever


# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
# print(f"Retriever initialisé pour récupérer les {retriever.search_kwargs['k']} documents les plus pertinents.")


print("\n--- Étape 6 : Configuration du LLM (Gemini) ---")
print("Chargement du prompt RAG depuis Langchain Hub...")
prompt = hub.pull("rlm/rag-prompt")
print("Prompt chargé.")

print("Initialisation du LLM ChatGoogleGenerativeAI (Gemini-Pro)...")
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.2)
    print("LLM Gemini-Pro initialisé avec succès.")
except Exception as e:
    print(f"ERREUR lors de l'initialisation du LLM ChatGoogleGenerativeAI : {e}")
    print("Assurez-vous que GOOGLE_API_KEY est correctement définie dans les secrets de Colab.")
    print("Vérifiez aussi que l'API Generative Language est activée pour votre projet Google Cloud.")
    exit("Processus arrêté car le LLM n'a pas pu être initialisé.")


print("\n--- Étape 7 : Construction de la Chaîne RAG ---")

# # --- NOUVEAU : Chaîne de Réécriture de Requête ---
# print("\n--- Étape 6.5 : Chaîne de Réécriture de Requête ---")
# query_rewriter_template = """You are an expert at rephrasing questions to be optimal for retrieving relevant documents.
# Given the following user question, rephrase it to be a standalone search query.
# If the question is already a good search query, return it as is.

# User question: {question}
# Rephrased query:"""

# query_rewriter_prompt = ChatPromptTemplate.from_template(query_rewriter_template)
# query_rewriting_chain = query_rewriter_prompt | llm | StrOutputParser()
# print("Chaîne de réécriture de la question initialisée.")

# --- : Chaîne de Génération de Requêtes Multiples --- MULTI_QUERY, RAG FUSION
print("\n--- Étape 6.5 : Chaîne de Transformation de Requête Multi-Requêtes ---")
multi_query_template = """You are an AI assistant that generates multiple search queries based on a single input question.
Your goal is to create diverse queries that cover various aspects of the user's original question,
to improve the chances of retrieving relevant documents.
Generate 3 distinct search queries related to the user's question, separated by newlines.
Do not number the queries.

Original question: {question}
Search queries:"""

multi_query_prompt = ChatPromptTemplate.from_template(multi_query_template)
multi_query_chain = multi_query_prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
print("Chaîne de génération de requêtes multiples initialisée.")

# --- DÉFINITION FINALE DE LA CHAÎNE RAG AVEC MULTI-QUERY ET SELF-QUERYING ---
print("\n--- Étape 7 : Construction de la Chaîne RAG ---")

# Fonction de formatage des documents
def format_docs_with_sources(docs: List[Document]) -> str:
    formatted_content = ""
    unique_pages = set()

    for i, doc in enumerate(docs):
        formatted_content += f"Contenu source {i+1}:\n{doc.page_content}\n\n"
        if 'page' in doc.metadata:
            page_number = doc.metadata['page'] + 1
            unique_pages.add(str(page_number))

    if unique_pages:
        formatted_content += f"Sources des pages: {', '.join(sorted(list(unique_pages)))}\n"

    return formatted_content

# # --- NOUVELLE DÉFINITION DE LA CHAÎNE RAG AVEC RÉÉCRITURE ET RECHERCHE HYBRIDE ---
# # Prépare les inputs : la question originale et la question réécrite pour le retriever
# prepare_inputs = {
#     "question_originale": RunnablePassthrough(),
#     "question_pour_retrieval": query_rewriting_chain,
# }

# # La chaîne de récupération utilise la question réécrite et passe le contexte + la question originale
# retrieval_chain_with_rewriting = {
#     "context": (lambda x: x["question_pour_retrieval"]) | retriever, # Le retriever utilise la question réécrite
#     "question": (lambda x: x["question_originale"]) # Le LLM utilise la question originale
# }

# # La chaîne de génération de réponse
# generation_chain = prompt | llm | StrOutputParser()

# # La chaîne RAG finale
# # L'entrée de rag_chain_with_sources est la question originale.
# # prepare_inputs prend cette question et la transforme en dictionnaire.
# # retrieval_chain_with_rewriting prend ce dictionnaire et utilise les clés "question_pour_retrieval" et "question_originale".
# rag_chain_with_sources = (
#     prepare_inputs
#     | RunnableParallel(
#         response=retrieval_chain_with_rewriting | generation_chain,
#         source_documents=retrieval_chain_with_rewriting | RunnableLambda(lambda x: x["context"])
#     )
# ).with_config(run_name="RAG Chain with Query Transformation and Sources")

# print("Chaîne RAG mise à jour avec la transformation de requête et la recherche hybride, prête à inclure les sources.")

# 1. Préparation des inputs: la question originale et la liste des questions générées
prepare_inputs = {
    "question_originale": RunnablePassthrough(), # La question initiale de l'utilisateur
    "questions_pour_retrieval": multi_query_chain, # La liste des requêtes générées
}

# 2. Chaîne de récupération : pour chaque requête générée, appeler le retriever, puis aplatir et dédupliquer
# Note: 'retriever' est maintenant l'EnsembleRetriever (BM25 + Self-Querying)
# retrieval_and_aggregation_chain = (
#     RunnableLambda(lambda x: x["questions_pour_retrieval"]) # Prend la liste de questions
#     | RunnableLambda(lambda queries: [retriever.invoke(q) for q in queries]) # Invoque l'EnsembleRetriever pour chaque requête
#     | RunnableLambda(lambda lists_of_docs: [doc for sublist in lists_of_docs for doc in sublist]) # Aplatit la liste de listes de docs
#     | RunnableLambda(lambda docs: list(set(docs))) # Supprime les doublons de Document
# )

# Déduplication des documents
retrieval_and_aggregation_chain = (
    RunnableLambda(lambda x: x["questions_pour_retrieval"])
    | RunnableLambda(lambda queries: [retriever.invoke(q) for q in queries])
    | RunnableLambda(lambda lists_of_docs: [doc for sublist in lists_of_docs for doc in sublist])
    # Logique pour la déduplication :
    | RunnableLambda(
        lambda docs: list(
            {
                (doc.page_content, frozenset(doc.metadata.items())): doc
                for doc in docs
            }.values()
        )
    )
)

# 3. Construction de la chaîne qui fournira le contexte et la question au LLM
# Elle prend en entrée le dictionnaire de 'prepare_inputs'
context_and_question_for_llm = {
    "context": retrieval_and_aggregation_chain, # Le contexte est le résultat agrégé du retrieval
    "question": RunnableLambda(lambda x: x["question_originale"]) # La question pour le LLM (la question l'originale)
}

# 4. Chaîne de génération de réponse (prompt | llm | output_parser)
generation_chain = prompt | llm | StrOutputParser()

# 5. La chaîne RAG finale
rag_chain_with_sources = (
    prepare_inputs # L'entrée est la question de l'utilisateur
    | RunnableParallel( # Exécute la génération de réponse et la récupération des sources en parallèle
        response=context_and_question_for_llm | generation_chain, # La branche qui génère la réponse
        source_documents=context_and_question_for_llm | RunnableLambda(lambda x: x["context"]) # La branche qui passe les documents sources
    )
).with_config(run_name="RAG Chain with Multi-Query & Self-Query")

print("Chaîne RAG construite avec succès : Multi-Query Retrieval, Self-Querying, et fusion des rags.")

print("\n--- Étape 8 : Tester la Chaîne RAG ---")

# question = "Quelles sont les fonctions du code du travail? Et quelles sont les conditions de représentativité des organisations syndicales ?"
# Testons une question plus conversationnelle pour voir la réécriture en action
# question_conv = "Quelle était la réforme mentionnée au début du document ? Et les conditions de représentativité ?"

question_conv = "Salut"
question = question_conv

print(f"\nQuestion : {question}")

try:
    result = rag_chain_with_sources.invoke(question)

    print("\nRéponse Générée par Gemini :")
    print(result["response"])

    # print("\nSources (extraits et métadonnées) :")
    # print(format_docs_with_sources(result["source_documents"]))

except Exception as e:
    print(f"ERREUR lors de l'invocation de la chaîne RAG : {e}")
    print("Veuillez vérifier les étapes précédentes (authentification LLM, initialisation de Chroma).")
    print("Erreur détaillée:", e)



--- Étape 4 : Création des Embeddings ---
Initialisation du modèle d'embedding : all-MiniLM-L6-v2
Modèle d'embedding initialisé avec succès.

--- Étape 5 : Création ou chargement du Vectorstore Chroma ---
Chargement du vectorstore Chroma existant depuis : ./chroma_db_codetravail
Vectorstore Chroma chargé.

--- Étape 5.5 : Configuration des Retrievers Hybrides ---
Retrievers hybrides (BM25 et Sémantique) configurés avec succès.
La recherche utilisera la fusion des rangs.

--- Étape 6 : Configuration du LLM (Gemini) ---
Chargement du prompt RAG depuis Langchain Hub...
Prompt chargé.
Initialisation du LLM ChatGoogleGenerativeAI (Gemini-Pro)...
LLM Gemini-Pro initialisé avec succès.

--- Étape 7 : Construction de la Chaîne RAG ---

--- Étape 6.5 : Chaîne de Transformation de Requête Multi-Requêtes ---
Chaîne de génération de requêtes multiples initialisée.

--- Étape 7 : Construction de la Chaîne RAG ---
Chaîne RAG construite avec succès : Multi-Query Retrieval, Self-Querying, et fusion d

# Application streamlit


In [ ]:
# # Installation des dépendances nécessaires
# !pip install -q langchain-google-genai pypdf langchain-chroma faiss-cpu sentence-transformers streamlit langchain_huggingface
# !pip install langchain_community langchainhub chromadb langchain langchain_chroma rank_bm25
# !pip install -q python-dotenv
# !pip install -q transformers
# !pip install -q --upgrade langchain

# print("Toutes les dépendances ont été installées ou mises à jour.")

# # Installation de localtunnel (pour exposer l'application)
# !apt-get update
# !apt-get install -y nodejs npm
# !npm install -g localtunnel

# print("Localtunnel installé.")

# # Configuration des clés API via Google Colab Secrets
# from google.colab import userdata
# import os

# try:
#     os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
#     os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
#     os.environ['LANGCHAIN_TRACING_V2'] = 'true'
#     os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
#     print("Clés API chargées depuis les secrets Colab.")
# except Exception as e:
#     print(f"ATTENTION : Erreur lors du chargement des clés API depuis les secrets Colab : {e}")
#     print("Assurez-vous que 'GOOGLE_API_KEY' et 'LANGCHAIN_API_KEY' sont bien définies dans les secrets et activées pour ce notebook.")

# from google.colab import drive
# drive.mount('/content/drive')
# print("Google Drive monté.")

# # Vérification du chemin du PDF
# pdf_path = "/content/drive/MyDrive/Colab Notebooks/AI31/lo17_rag/data/data_50_page.pdf"
# if not os.path.exists(pdf_path):
#     print(f"ERREUR CRITIQUE : Le fichier PDF '{pdf_path}' n'a pas été trouvé. Veuillez vérifier le chemin.")
# else:
#     print(f"Chemin du PDF vérifié : {pdf_path}")

In [ ]:
# # %%writefile app.py

# import streamlit as st
# import os
# import re
# from typing import List, Optional, Dict, Any
# from langchain.text_splitter import TextSplitter
# from langchain.schema import Document
# from langchain_community.document_loaders import PyPDFLoader
# from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain import hub
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_community.retrievers import BM25Retriever
# from langchain.retrievers import EnsembleRetriever
# import sys

# # --- Configuration Streamlit (DOIT être la première commande Streamlit) ---
# st.set_page_config(page_title="Assistant Juridique RAG (Code du Travail)", layout="wide")

# # --- Chemins et configuration ---
# pdf_path = "/content/drive/MyDrive/Colab Notebooks/AI31/lo17_rag/data/data_50_page.pdf"
# # pdf_path = "data/data_50_page.pdf"
# chroma_db_dir = "./chroma_db_codetravail"
# embedding_model_name = "all-MiniLM-L6-v2"

# # --- Interface Streamlit ---
# st.title("⚖️ Assistant Juridique")
# st.markdown("""
# Bienvenue dans votre outil d'aide à la décision juridique. Posez des questions sur le Code du Travail
# et obtenez des réponses précises basées sur les documents officiels.
# """)

# st.write("Chaîne RAG construite avec succès : Multi-Query Retrieval, Self-Querying, et fusion des rags.")

# if "params" not in st.session_state:
#     st.session_state.params = {
#         "temperature": 0.2,
#         "top_p": 10,
#         "model": "gemini-2.5-flash-preview-05-20"
#     }

# # --- Classe TitleBasedSplitter ---
# class TitleBasedSplitter(TextSplitter):
#     def __init__(self, pattern: str = r"(Titre\s+(?:[IVXLCDM]+(?:er|ème)?|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:)"):
#         super().__init__()
#         self.compiled_pattern = re.compile(pattern, re.IGNORECASE)

#     def split_documents(self, documents: List[Document], **kwargs) -> List[Document]:
#         all_chunks: List[Document] = []
#         for doc in documents:
#             text = doc.page_content
#             metadata = doc.metadata.copy()

#             matches = list(self.compiled_pattern.finditer(text))

#             if not matches:
#                 if text.strip():
#                     all_chunks.append(Document(page_content=text.strip(), metadata=metadata))
#                 continue

#             if matches[0].start() > 0:
#                 pre_title_text = text[0:matches[0].start()].strip()
#                 if pre_title_text:
#                     all_chunks.append(Document(page_content=pre_title_text, metadata=metadata))

#             for i in range(len(matches)):
#                 start = matches[i].start()
#                 end = matches[i+1].start() if i + 1 < len(matches) else len(text)
#                 chunk_content = text[start:end].strip()

#                 if chunk_content:
#                     chunk_metadata = metadata.copy()
#                     all_chunks.append(Document(page_content=chunk_content, metadata=chunk_metadata))
#         return all_chunks

#     def split_text(self, text: str) -> List[str]:
#         chunks = []
#         matches = list(self.compiled_pattern.finditer(text))
#         if not matches:
#             return [text.strip()] if text.strip() else []

#         if matches[0].start() > 0:
#             pre_title_text = text[0:matches[0].start()].strip()
#             if pre_title_text:
#                 chunks.append(pre_title_text)

#         for i in range(len(matches)):
#             start = matches[i].start()
#             end = matches[i+1].start() if i + 1 < len(matches) else len(text)
#             chunk_content = text[start:end].strip()
#             if chunk_content:
#                 chunks.append(chunk_content)
#         return chunks

# # --- Classe CodeDuTravailStructureExtractor ---
# class CodeDuTravailStructureExtractor(TextSplitter):
#     def __init__(
#         self,
#         title_pattern: str = r"(Titre\s+(?:[IVXLCDM]+(?:er|ème)?|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
#         chapter_pattern: str = r"(Chapitre\s+(?:[IVXLCDM]+(?:er|ème)?|unique|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
#         section_pattern: str = r"(Section\s+(?:\d+|unique|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
#         article_pattern: str = r"(Article\s+((?:L|R|D)\.\s*\d{3}-\d+(?:-\d+)?(?:-\d+)?)[\s\S]*?(?=Article\s+((?:L|R|D)\.\s*\d{3}-\d+(?:-\d+)?(?:-\d+)?)|Titre\s+|Chapitre\s+|Section\s+|$))",
#         article_num_capture_group: int = 2,
#         keep_separator: bool = True,
#         **kwargs
#     ):
#         super().__init__(keep_separator=keep_separator, **kwargs)
#         self.title_pattern = re.compile(title_pattern, re.IGNORECASE | re.DOTALL)
#         self.chapter_pattern = re.compile(chapter_pattern, re.IGNORECASE | re.DOTALL)
#         self.section_pattern = re.compile(section_pattern, re.IGNORECASE | re.DOTALL)
#         self.article_pattern = re.compile(article_pattern, re.IGNORECASE | re.DOTALL)
#         self.article_num_capture_group = article_num_capture_group

#     def split_documents(self, documents: List[Document]) -> List[Document]:
#         all_chunks: List[Document] = []
#         current_book = None
#         current_part = None
#         current_title = None
#         current_chapter = None
#         current_section = None

#         for doc in documents:
#             text = doc.page_content
#             page_metadata = doc.metadata.copy()

#             article_matches = list(self.article_pattern.finditer(text))

#             last_idx = 0
#             if article_matches and article_matches[0].start() > 0:
#                 pre_article_text = text[0:article_matches[0].start()].strip()
#                 if pre_article_text:
#                     chunk_metadata = self._extract_hierarchy_metadata(pre_article_text, page_metadata)
#                     all_chunks.append(Document(page_content=pre_article_text, metadata=chunk_metadata))
#                 last_idx = article_matches[0].start()

#             for i in range(len(article_matches)):
#                 article_content = article_matches[i].group(0).strip()
#                 article_number = article_matches[i].group(self.article_num_capture_group)

#                 chunk_metadata = page_metadata.copy()
#                 chunk_metadata["type"] = "Article"
#                 chunk_metadata["article_number"] = article_number

#                 chunk_metadata.update(self._extract_hierarchy_metadata(article_content, page_metadata))

#                 all_chunks.append(Document(page_content=article_content, metadata=chunk_metadata))
#                 last_idx = article_matches[i].end()

#             if last_idx < len(text):
#                 remaining_text = text[last_idx:].strip()
#                 if remaining_text:
#                     chunk_metadata = self._extract_hierarchy_metadata(remaining_text, page_metadata)
#                     all_chunks.append(Document(page_content=remaining_text, metadata=chunk_metadata))
#         return all_chunks


#     def _extract_hierarchy_metadata(self, text: str, base_metadata: Dict[str, Any]) -> Dict[str, Any]:
#         metadata = base_metadata.copy()

#         title_match = self.title_pattern.search(text)
#         if title_match:
#             metadata["title"] = title_match.group(1).strip()

#         chapter_match = self.chapter_pattern.search(text)
#         if chapter_match:
#             metadata["chapter"] = chapter_match.group(1).strip()

#         section_match = self.section_pattern.search(text)
#         if section_match:
#             metadata["section"] = section_match.group(1).strip()

#         return metadata

#     def split_text(self, text: str) -> List[str]:
#         chunks = []
#         article_splits = self.article_pattern.split(text)

#         if len(article_splits) > 0 and not self.article_pattern.match(article_splits[0]):
#             if article_splits[0].strip():
#                 chunks.append(article_splits[0].strip())
#             start_index = 1
#         else:
#             start_index = 0

#         for i in range(start_index, len(article_splits), self.article_num_capture_group + 1):
#             if i + self.article_num_capture_group < len(article_splits):
#                 article_num = article_splits[i + self.article_num_capture_group -1]
#                 article_content = article_splits[i + self.article_num_capture_group]
#                 full_article_chunk = f"Article {article_num.strip()} {article_content.strip()}"
#                 if full_article_chunk.strip():
#                     chunks.append(full_article_chunk.strip())
#             elif article_splits[i].strip():
#                 chunks.append(article_splits[i].strip())
#         return chunks

# # --- Chargement et Traitement du PDF ---
# documents = []
# if not os.path.exists(pdf_path):
#     st.error(f"ERREUR: Le fichier PDF '{pdf_path}' n'a pas été trouvé. Veuillez vérifier le chemin.")
#     st.stop() # Arrête l'exécution de l'application Streamlit
# else:
#     st.write(f"Chargement du PDF depuis : {pdf_path}")
#     loader = PyPDFLoader(pdf_path)
#     docs_from_loader = loader.load()

#     if not docs_from_loader:
#         st.error("Aucun document n'a pu être chargé par PyPDFLoader. Veuillez vérifier le PDF.")
#         st.stop() # Arrête l'exécution de l'application Streamlit
#     else:
#         st.write(f"Nombre de pages chargées par PyPDFLoader : {len(docs_from_loader)}")
#         structure_extractor = CodeDuTravailStructureExtractor()
#         documents = structure_extractor.split_documents(docs_from_loader)
#         st.write(f"Nombre de documents (chunks enrichis) après extraction de structure : {len(documents)}")

# if not documents:
#     st.error("Le traitement ne peut pas continuer car aucun document structuré n'a été créé.")
#     st.stop() # Arrête l'exécution de l'application Streamlit

# # --- Création des Embeddings ---
# try:
#     embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
# except Exception as e:
#     st.error(f"ERREUR lors de l'initialisation du modèle d'embedding : {e}")
#     st.info("Veuillez vérifier votre connexion internet et les dépendances 'sentence-transformers'.")
#     st.stop() # Arrête l'exécution de l'application Streamlit

# # --- Création ou chargement du Vectorstore Chroma ---
# try:
#     if os.path.exists(chroma_db_dir) and os.listdir(chroma_db_dir):
#         try:
#             vectorstore = Chroma(persist_directory=chroma_db_dir, embedding_function=embedding_model)
#             st.success("Base de données vectorielle existante chargée avec succès.")
#         except Exception as e:
#             st.warning(f"Erreur lors du chargement de la base existante : {e}. Création d'une nouvelle base...")
#             vectorstore = Chroma.from_documents(documents, embedding=embedding_model, persist_directory=chroma_db_dir)
#             st.success("Nouvelle base de données vectorielle créée avec succès.")
#     else:
#         vectorstore = Chroma.from_documents(documents, embedding=embedding_model, persist_directory=chroma_db_dir)
#         st.success("Nouvelle base de données vectorielle créée avec succès.")
# except Exception as e:
#     st.error(f"Erreur critique lors de l'initialisation de ChromaDB : {e}")
#     st.info("Veuillez vérifier que tous les packages sont correctement installés :")
#     st.code("pip install -U langchain-huggingface langchain-chroma chromadb jsonschema")
#     st.stop()

# # --- Configuration des Retrievers Hybrides ---
# semantic_retriever = vectorstore.as_retriever(search_kwargs={"k": st.session_state.params['top_p']})
# bm25_retriever = BM25Retriever.from_documents(documents)
# bm25_retriever.k = 5
# hybrid_retriever = EnsembleRetriever(
#     retrievers=[bm25_retriever, semantic_retriever],
#     weights=[0.5, 0.5]
# )
# retriever = hybrid_retriever
# st.write("Retrievers hybrides (BM25 et Sémantique) configurés.")

# # --- Configuration du LLM (Gemini) ---
# prompt = hub.pull("rlm/rag-prompt")
# try:
#     llm = ChatGoogleGenerativeAI(
#         model=st.session_state.params['model'],
#         temperature=st.session_state.params['temperature'])
# except Exception as e:
#     st.error(f"ERREUR lors de l'initialisation du LLM ChatGoogleGenerativeAI : {e}")
#     st.info("Assurez-vous que GOOGLE_API_KEY est correctement définie dans les secrets de Colab et que l'API Generative Language est activée.")
#     st.stop()

# # --- Chaîne de Génération de Requêtes Multiples ---
# multi_query_template = """You are an AI assistant that generates multiple search queries based on a single input question. Your goal is to create diverse queries that cover various aspects of the user's original question, to improve the chances of retrieving relevant documents. Generate 3 distinct search queries related to the user's question, separated by newlines. Do not number the queries. Original question: {question} Search queries:"""
# multi_query_prompt = ChatPromptTemplate.from_template(multi_query_template)
# multi_query_chain = multi_query_prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
# st.write("Chaîne de génération de requêtes multiples initialisée.")

# # --- DÉFINITION FINALE DE LA CHAÎNE RAG AVEC MULTI-QUERY ET SELF-QUERYING ---
# def format_docs_with_sources(docs: List[Document]) -> str:
#     formatted_content = ""
#     unique_pages = set()

#     for i, doc in enumerate(docs):
#         formatted_content += f"Contenu source {i+1}:\n{doc.page_content}\n\n"
#         if 'page' in doc.metadata:
#             page_number = doc.metadata['page'] + 1
#             unique_pages.add(str(page_number))

#     if unique_pages:
#         formatted_content += f"Sources des pages: {', '.join(sorted(list(unique_pages)))}\n"
#     return formatted_content

# prepare_inputs = {
#     "question_originale": RunnablePassthrough(),
#     "questions_pour_retrieval": multi_query_chain,
# }

# retrieval_and_aggregation_chain = (
#     RunnableLambda(lambda x: x["questions_pour_retrieval"])
#     | RunnableLambda(lambda queries: [retriever.invoke(q) for q in queries])
#     | RunnableLambda(lambda lists_of_docs: [doc for sublist in lists_of_docs for doc in sublist])
#     | RunnableLambda(
#         lambda docs: list(
#             {
#                 (doc.page_content, frozenset(doc.metadata.items())): doc
#                 for doc in docs
#             }.values()
#         )
#     )
# )

# context_and_question_for_llm = {
#     "context": retrieval_and_aggregation_chain,
#     "question": RunnableLambda(lambda x: x["question_originale"])
# }

# generation_chain = prompt | llm | StrOutputParser()

# rag_chain_with_sources = (
#     prepare_inputs
#     | RunnableParallel(
#         response=context_and_question_for_llm | generation_chain,
#         source_documents=context_and_question_for_llm | RunnableLambda(lambda x: x["context"])
#     )
# ).with_config(run_name="RAG Chain with Multi-Query & Self-Query")


# if "messages" not in st.session_state:
#     st.session_state.messages = []

# for message in st.session_state.messages:
#     with st.chat_message(message["role"]):
#         st.markdown(message["content"])

# if prompt_input := st.chat_input("Posez votre question sur le Code du Travail..."):
#     st.session_state.messages.append({"role": "user", "content": prompt_input})
#     with st.chat_message("user"):
#         st.markdown(prompt_input)

#     with st.chat_message("assistant"):
#         with st.spinner("Recherche de réponse..."):
#             full_response = rag_chain_with_sources.invoke(prompt_input)
#             st.markdown(full_response["response"]) # Accéder à la clé 'response'

#             # Afficher les sources
#             if "source_documents" in full_response and full_response["source_documents"]:
#                 st.subheader("Sources:")
#                 st.markdown(format_docs_with_sources(full_response["source_documents"]))

#         st.session_state.messages.append({"role": "assistant", "content": full_response["response"]}) # Stocker la réponse
#         # Optionnel: stocker aussi les sources si tu veux les afficher à chaque re-run
#         # st.session_state.messages.append({"role": "assistant_sources", "content": format_docs_with_sources(full_response["source_documents"])})


# with st.sidebar:
#     st.header("Options")
#     st.subheader("paramètre du modèle")

#     model = st.selectbox(
#         "Modèle Gemini",
#         options=[
#             "gemini-2.5-flash-preview-05-20",
#             "gemini-1.5-pro",
#             "gemini-1.5-flash"
#         ],
#         index=0,
#         help="Choisissez le modèle Gemini à utiliser"
#     )

#     temperature = st.slider(
#         "Temperature",
#         min_value=0.0,
#         max_value=1.0,
#         value=st.session_state.params['temperature'],
#         step=0.1,
#         help="Contrôle la créativité des réponses (0 = plus précis, 1 = plus créatif)"
#     )
#     top_p = st.slider(
#         "Top_p",
#         min_value=1,
#         max_value=15,
#         value=st.session_state.params['top_p'],
#         step=1,
#         help="Nombre de documents à récupérer pour la réponse"
#     )

#     st.session_state.params['model']=model
#     st.session_state.params['temperature'] = temperature
#     st.session_state.params['top_p'] = top_p

#     if st.button("Effacer l'historique du chat"):
#         st.session_state.messages = []
#         st.rerun()

In [ ]:
# import subprocess
# import time
# import re

# print("Lancement de l'application Streamlit en arrière-plan...")
# # Lance Streamlit en arrière-plan sur le port 8501
# streamlit_process = subprocess.Popen([
#     "streamlit", "run", "app.py",
#     "--server.port", "8501",
#     "--server.enableCORS", "false",
#     "--server.enableXsrfProtection", "false"
# ])

# print("Attente du démarrage de Streamlit (environ 5 secondes)...")
# time.sleep(5) # Donne à Streamlit le temps de démarrer

# print("Création du tunnel localtunnel...")
# # Lance localtunnel pour exposer le port 8501
# localtunnel_process = subprocess.Popen(['lt', '--port', '8501'], stdout=subprocess.PIPE, text=True)

# public_url = None
# for _ in range(30):
#     line = localtunnel_process.stdout.readline()
#     print(f"localtunnel output: {line.strip()}")
#     match = re.search(r'https?://[^\s/$.?#].[^\s]*\.loca\.lt', line)
#     if match:
#         public_url = match.group(0)
#         break
#     time.sleep(1)

# if public_url:
#     print(f"🎉 Votre application Streamlit est disponible à l'adresse : {public_url}")
# else:
#     print("Erreur : Impossible d'obtenir l'URL de localtunnel. Vérifie la sortie ci-dessus pour les erreurs.")
#     print("Vérifie si localtunnel est bien installé (`!npm install -g localtunnel`) et si Streamlit a démarré.")
#     streamlit_process.terminate() # Tente de tuer le processus Streamlit si localtunnel échoue
#     localtunnel_process.terminate() # Tente de tuer le processus localtunnel



In [ ]:
# if 'streamlit_process' in locals() and streamlit_process.poll() is None:
#     streamlit_process.terminate()
#     print("Processus Streamlit terminé.")
# if 'localtunnel_process' in locals() and localtunnel_process.poll() is None:
#     localtunnel_process.terminate()
#     print("Processus localtunnel terminé.")

In [ ]:
# !curl https://loca.lt/mytunnelpassword

# Evaluation

In [ ]:
from typing import List, Dict, Any

evaluation_dataset = [
    {
        "question": "
        "ground_truth_answer": "La loi n° 2008-67 du 21 janvier 2008 prévoit une concertation préalable obligatoire pour tout projet de réforme du Gouvernement concernant les relations individuelles et collectives du travail, l'emploi et la formation professionnelle qui relève de la négociation nationale et interprofessionnelle. Cette concertation s'adresse aux organisations syndicales de salariés et d'employeurs représentatives au niveau national et interprofessionnel. Le Gouvernement doit leur communiquer un document d'orientation. Cette procédure n'est pas applicable en cas d'urgence.",
        "ground_truth_relevant_document_pages": [0] # Page 1 du document
    },
    {
        "question": "Quels sont les sujets principaux de la négociation triennale sur l'égalité professionnelle suite à l'ordonnance de 2017 ?",
        "ground_truth_answer": "L'ordonnance n° 2017-1385 du 22 septembre 2017 a modifié la négociation triennale sur l'égalité professionnelle entre les femmes et les hommes. Cette négociation porte notamment sur les conditions d'accès à l'emploi, à la formation et à la promotion professionnelle, ainsi que sur les conditions de travail et d'emploi, en particulier celles des salariés à temps partiel. Lorsque les mesures de rattrapage portent sur des mesures salariales, leur mise en œuvre est suivie dans le cadre de la négociation annuelle obligatoire sur les salaires.",
        "ground_truth_relevant_document_pages": [277] # Page 278 du document
    }
    # {
    #     "question": "Énumérez les sept critères légaux de représentativité des organisations syndicales de salariés.",
    #     "ground_truth_answer": "Les sept critères légaux de représentativité des organisations syndicales de salariés sont : 1) Le respect des valeurs républicaines, 2) L'indépendance, 3) La transparence financière, 4) Une ancienneté minimale de deux ans dans le champ professionnel et géographique de la négociation, 5) L'audience (mesurée par les résultats aux élections professionnelles), 6) L'influence (caractérisée par l'activité et l'expérience), 7) Les effectifs d'adhérents et les cotisations.",
    #     "ground_truth_relevant_document_pages": [25] # Page 25 du document (Basé sur L. 2121-1)
    # },
    # {
    #     "question": "Quelle est la mission des inspecteurs du travail et quel article du Code du travail la définit ?",
    #     "ground_truth_answer": "Les inspecteurs du travail ont pour mission de veiller à l'application des dispositions légales relatives au régime du travail. Ils exercent un rôle de contrôle et de conseil. L'article L. 8112-1 du Code du travail définit cette mission.",
    #     "ground_truth_relevant_document_pages": [10] # Page 10 du document (Basé sur L. 8112-1)
    # },
    # {
    #     "question": "En dehors de l'égalité professionnelle, quels autres sujets sont abordés dans la négociation triennale ?",
    #     "ground_truth_answer": "Outre l'égalité professionnelle, la négociation triennale aborde également les conditions de travail et la gestion prévisionnelle des emplois et des parcours professionnels (GPEC), notamment au travers du paragraphe 2 de l'article L. 2241-11 sur les conditions de travail et gestion prévisionnelle des emplois.",
    #     "ground_truth_relevant_document_pages": [5] # Page 5 du document
    # },
    # {
    #     "question": "La loi n° 2020-1525 du 7 décembre 2020 est mentionnée en page 1. Quel est son impact sur le dialogue social ?",
    #     "ground_truth_answer": "La loi n° 2020-1525 du 7 décembre 2020 est mentionnée en page 1 en lien avec le chapitre préliminaire sur le dialogue social. Cependant, les extraits de la page 1 ne décrivent pas en détail l'impact spécifique de cette loi sur le dialogue social, mais la situent dans ce contexte.",
    #     "ground_truth_relevant_document_pages": [1] # Page 1 du document
    # },
    # {
    #     "question": "Quand le Gouvernement peut-il déroger à la procédure de concertation préalable pour une réforme du travail, et que doit-il faire dans ce cas ?",
    #     "ground_truth_answer": "Le présent article (L.1) n'est pas applicable en cas d'urgence. Lorsque le Gouvernement décide de mettre en œuvre un projet de réforme en l'absence de procédure de concertation en raison de l'urgence, il doit faire connaître cette décision aux organisations syndicales et d'employeurs en la motivant dans un document transmis avant de prendre toute mesure.",
    #     "ground_truth_relevant_document_pages": [1] # Page 1 du document
    # },
    # {
    #     "question": "Quelles informations sont nécessaires à la négociation sur l'égalité professionnelle ?",
    #     "ground_truth_answer": "Les informations nécessaires à la négociation sur l'égalité professionnelle entre les femmes et les hommes sont déterminées par voie réglementaire.",
    #     "ground_truth_relevant_document_pages": [5] # Page 5 du document
    # },
    # {
    #     "question": "Qu'est-ce que l'audience électorale dans le cadre de la représentativité syndicale, et à quel article fait-elle référence pour la négociation annuelle obligatoire ?",
    #     "ground_truth_answer": "L'audience électorale est un critère de représentativité des organisations syndicales, mesuré notamment lors des élections professionnelles. La mise en œuvre des mesures de rattrapage salariales issues de la négociation sur l'égalité professionnelle est suivie dans le cadre de la négociation annuelle obligatoire sur les salaires, prévue à l'article L. 2241-8 du Code du travail.",
    #     "ground_truth_relevant_document_pages": [5, 25] # Pages 5 et 25 (exemple de multi-sources)
    # },
    # {
    #     "question": "Quelle est l'importance de la transparence financière pour une organisation syndicale ?",
    #     "ground_truth_answer": "La transparence financière est l'un des sept critères de représentativité des organisations syndicales de salariés, essentiel pour garantir leur légitimité et leur intégrité.",
    #     "ground_truth_relevant_document_pages": [25] # Page 25 du document
    # },
    # {
    #     "question": "Comment la loi de 2008 et l'ordonnance de 2017 ont-elles affecté le Code du travail concernant le dialogue social et l'égalité professionnelle ?",
    #     "ground_truth_answer": "La loi n° 2008-67 du 21 janvier 2008 a introduit des dispositions concernant le chapitre préliminaire sur le dialogue social, notamment la concertation préalable pour les réformes du Gouvernement. L'ordonnance n° 2017-1385 du 22 septembre 2017, quant à elle, a modifié les mesures tendant à assurer l'égalité professionnelle entre les femmes et les hommes, impactant les négociations triennales sur ce sujet, y compris les conditions d'accès à l'emploi et de travail.",
    #     "ground_truth_relevant_document_pages": [1, 5] # Exemple de question nécessitant de multiples sources
    # }
]

print(f"Jeu de données d'évaluation créé avec {len(evaluation_dataset)} questions.")

Jeu de données d'évaluation créé avec 2 questions.


In [ ]:
!pip install -qU ragas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
!pip show ragas datasets

Name: ragas
Version: 0.2.15
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: appdirs, datasets, diskcache, langchain, langchain-community, langchain-core, langchain_openai, nest-asyncio, numpy, openai, pydantic, tiktoken
Required-by: 
---
Name: datasets
Version: 3.6.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: ragas, torchtune


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda # Pour la conversion de la chaîne en liste

from langchain_core.documents import Document

def evaluate_retrieval(retriever_instance, eval_data: List[Dict]):
    """
    Évalue les métriques de récupération (Hit Rate, Recall, MRR) d'un retriever.

    Args:
        retriever_instance: L'instance du retriever LangChain à évaluer (ton hybrid_retriever).
        eval_data: Le jeu de données d'évaluation.
    """
    hit_rates = []
    recalls_at_k = []
    mrr_scores = []

    print("\n--- Évaluation du Retrieval ---")
    for i, item in enumerate(eval_data):
        question = item["question"]
        ground_truth_pages = set(item["ground_truth_relevant_document_pages"])

        try:
            # Il est primordial de s'assurer que la question est un dictionnaire si la chaîne d'entrée l'attend
            # Si le rag_chain_with_sources.invoke(question) prend une string,
            # alors le retriever.invoke() prendra aussi la string.
            # Cependant, avec Multi-Query, le retriever est appelé via la chaîne interne.
            # Pour l'évaluation du retriever seul, nous devons lui passer la question directe.
            # Ici, nous allons simuler l'appel à la chaîne de multi-query + retriever agrégé.

            # Étape de multi-query
            # L'on s'assure que multi_query_chain attend un dictionnaire comme entrée si elle fait partie d'une RunnableSequence
            generated_queries = multi_query_chain.invoke({"question": question})

            # Exécution de l'EnsembleRetriever pour chaque query et agrégation
            all_retrieved_docs = []
            for q_gen in generated_queries:
                # C'est ici que l'EnsembleRetriever (qui contient SelfQueryRetriever) est invoqué.
                # Il prend directement la requête en string.
                # Le retriever_instance est l'EnsembleRetriever
                retrieved_for_query = retriever_instance.invoke(q_gen)
                all_retrieved_docs.extend(retrieved_for_query)

            # Déduplication des documents récupérés
            # Un set de tuples (page_content, metadata_tuple) pour être sûr de la déduplication
            unique_docs = []
            seen_doc_ids = set() # Utilise un identifiant unique si disponible, sinon page_content
            for doc in all_retrieved_docs:
                doc_id = (doc.page_content, frozenset(doc.metadata.items())) # Crée un identifiant unique basé sur content + metadata
                if doc_id not in seen_doc_ids:
                    unique_docs.append(doc)
                    seen_doc_ids.add(doc_id)

            retrieved_docs = unique_docs

            # Extraction des pages des documents récupérés
            retrieved_pages = set([doc.metadata.get('page', -1) for doc in retrieved_docs if 'page' in doc.metadata])

            # --- Calcul des métriques ---
            # Hit Rate
            # Vérifie si AU MOINS une page récupérée est dans les pages attendues
            hit = any(p in ground_truth_pages for p in retrieved_pages)
            hit_rates.append(1 if hit else 0)

            # Recall@k (k est le nombre total de documents pertinents dans le ground truth)
            # Mesure la proportion de documents pertinents attendus qui ont été effectivement récupérés.
            if len(ground_truth_pages) > 0:
                relevant_retrieved = len(ground_truth_pages.intersection(retrieved_pages))
                recalls_at_k.append(relevant_retrieved / len(ground_truth_pages))
            else: # Si pas de documents pertinents attendus, recall est 1 si 0 documents sont récupérés
                recalls_at_k.append(1 if len(retrieved_pages) == 0 else 0)

            # MRR (Mean Reciprocal Rank)
            # Mesure où le premier document pertinent apparaît dans le classement.
            mrr_score = 0
            # On cherche la première page pertinente dans la liste ORDONNÉE des documents récupérés
            # L'ordre ici dépend de la fusion des rangs de l'EnsembleRetriever
            for rank, doc in enumerate(retrieved_docs):
                if doc.metadata.get('page', -1) in ground_truth_pages:
                    mrr_score = 1.0 / (rank + 1)
                    break
            mrr_scores.append(mrr_score)

            print(f"  Q{i+1}: '{question[:70]}...'")
            print(f"    GT Pages: {sorted(list(ground_truth_pages))}, Retrieved Pages: {sorted(list(retrieved_pages))}")
            print(f"    Hit: {hit}, Recall: {recalls_at_k[-1]:.2f}, MRR: {mrr_scores[-1]:.2f}")

        except Exception as e:
            print(f"  Erreur lors de l'évaluation de la question '{question[:70]}...': {e}")
            hit_rates.append(0)
            recalls_at_k.append(0)
            mrr_scores.append(0)


    print("\n--- Résultats Généraux du Retrieval ---")
    # On évite la division par zéro si eval_data est vide
    num_questions = len(eval_data)
    if num_questions > 0:
        print(f"Average Hit Rate: {sum(hit_rates) / num_questions:.2f}")
        print(f"Average Recall: {sum(recalls_at_k) / num_questions:.2f}")
        print(f"Average MRR: {sum(mrr_scores) / num_questions:.2f}")
    else:
        print("Aucune question dans le jeu de données d'évaluation.")


# --- Appel de la fonction d'évaluation après la définition de toutes les chaînes ---
# L'on s'assure que 'retriever' et 'multi_query_chain' sont définis avant cet appel.
evaluate_retrieval(retriever, evaluation_dataset)


# --- Intégration de RAGAS pour l'évaluation de la génération ---
# NB: ragas doit être bien installer - !pip install -q ragas datasets
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import (
    Faithfulness,
    ContextRelevance,
    AnswerCorrectness
)

def evaluate_generation_with_ragas(rag_chain, eval_data: List[Dict], llm_model, embedding_model_ragas):
    """
    Évalue la génération de la réponse RAG en utilisant RAGAS.
    """
    data_for_ragas = {
        "question": [],
        "answer": [],
        "contexts": [], # Liste de listes de strings (contenu des docs)
        "ground_truth": [] # Les réponses de référence humaines
    }

    print("\n--- Préparation des données pour RAGAS (exécution du RAG pour chaque question) ---")
    for i, item in enumerate(eval_data):
        question = item["question"]
        ground_truth_answer = item.get("ground_truth_answer", None)

        try:
            # L'entrée de rag_chain_with_sources est directement la string de la question
            result = rag_chain.invoke(question) # Appelle du RAG complet

            data_for_ragas["question"].append(question)
            data_for_ragas["answer"].append(result.get("response", "No response generated"))
            # ragas attend une liste de strings pour le contexte
            data_for_ragas["contexts"].append([doc.page_content for doc in result.get("source_documents", [])])
            data_for_ragas["ground_truth"].append(ground_truth_answer)
            print(f"  Processed Q{i+1}: '{question[:70]}...'")

        except Exception as e:
            print(f"  ERREUR lors de l'exécution du RAG pour la question '{question[:70]}...': {e}")
            import traceback
            traceback.print_exc()
            # Ajout des valeurs vides pour ne pas casser RAGAS
            data_for_ragas["question"].append(question)
            data_for_ragas["answer"].append("ERROR: RAG chain failed")
            data_for_ragas["contexts"].append([])
            data_for_ragas["ground_truth"].append(ground_truth_answer)

    # S'assurer qu'il y a des données avant de créer le Dataset
    if not data_for_ragas["question"]:
         print("Aucune donnée n'a pu être préparée pour RAGAS.")
         return

    ragas_dataset = Dataset.from_dict(data_for_ragas)

    print("\n--- Exécution de l'évaluation RAGAS ---")
    # Liste des métriques à évaluer
    metrics_to_evaluate = [
        Faithfulness,
        ContextRelevance,
        AnswerCorrectness
    ]

    try:
        result_ragas = evaluate(
            ragas_dataset,
            metrics=metrics_to_evaluate,
            llm=llm_model, # Utilisation du LLM pour les métriques basées sur LLM-as-a-Judge
            embeddings=embedding_model_ragas, # Utilisation du modèle d'embedding pour les métriques basées sur embedding-as-a-Judge
        )

        df_results_ragas = result_ragas.to_dataframe()
        print("\n--- Résultats RAGAS Moyens ---")
        print(df_results_ragas.mean(numeric_only=True))
        print("\n--- Détail des Résultats RAGAS ---")
        print(df_results_ragas)

    except Exception as e:
          print(f"ERREUR lors de l'exécution de l'évaluation RAGAS: {e}")
          import traceback
          traceback.print_exc() # Afficher la trace complète de l'erreur pour le debug


--- Évaluation du Retrieval ---
  Q1: 'Selon la loi de 2008, quelle est la procédure de concertation préalabl...'
    GT Pages: [0], Retrieved Pages: [0, 21, 22, 37, 61, 68, 78, 79, 85, 92, 107, 181, 182, 187, 222, 255, 256]
    Hit: True, Recall: 1.00, MRR: 1.00
  Q2: 'Quels sont les sujets principaux de la négociation triennale sur l'éga...'
    GT Pages: [277], Retrieved Pages: [12, 39, 114, 118, 121, 169, 170, 216, 217, 222, 228, 229, 253, 255, 261, 264, 272, 275, 276, 277]
    Hit: True, Recall: 1.00, MRR: 1.00

--- Résultats Généraux du Retrieval ---
Average Hit Rate: 1.00
Average Recall: 1.00
Average MRR: 1.00


In [ ]:
# Appel de la fonction d'évaluation RAGAS
# L'on s'assure que rag_chain_with_sources, llm, et embedding_model sont définis.

if 'rag_chain_with_sources' in globals() and 'llm' in globals() and 'embedding_model' in globals() and 'evaluation_dataset' in globals():
    evaluate_generation_with_ragas(rag_chain_with_sources, evaluation_dataset, llm, embedding_model)
else:
    print("Impossible d'exécuter l'évaluation RAGAS : variables nécessaires non définies.")


--- Préparation des données pour RAGAS (exécution du RAG pour chaque question) ---
  Processed Q1: 'Selon la loi de 2008, quelle est la procédure de concertation préalabl...'
  Processed Q2: 'Quels sont les sujets principaux de la négociation triennale sur l'éga...'

--- Exécution de l'évaluation RAGAS ---
ERREUR lors de l'exécution de l'évaluation RAGAS: 'property' object has no attribute 'get'


Traceback (most recent call last):
  File "<ipython-input-17-3883995461>", line 176, in evaluate_generation_with_ragas
    result_ragas = evaluate(
                   ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ragas/_analytics.py", line 227, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ragas/evaluation.py", line 176, in evaluate
    validate_required_columns(dataset, metrics)
  File "/usr/local/lib/python3.11/dist-packages/ragas/validation.py", line 60, in validate_required_columns
    required_columns = set(m.required_columns.get(metric_type, []))
                           ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'property' object has no attribute 'get'


In [ ]:
if documents:
    print(documents[0].metadata) # Regardez la clé 'page' pour le premier document
    print(documents[4].metadata) # Regardez la clé 'page' pour le cinquième document (si disponible)

{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-06-03T20:04:09+00:00', 'source': '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf', 'total_pages': 278, 'page': 0, 'page_label': '1', 'chapter': 'Chapitre préliminaire : Dialogue social. \nPartie législative'}
{'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-06-03T20:04:09+00:00', 'source': '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf', 'total_pages': 278, 'page': 4, 'page_label': '5'}


## UTILITE DU PROJET
1. **Qu'est-ce qu'une application RAG ? (Retrieval Augmented Generation)**
Imaginons une immense bibliothèque (dans notre cas, un PDF de 2775 pages sur le droit du travail) et qu'un utilisateur pose une question très spécifique.

**Sans RAG :** Si l'utilisateur posait cette question à un grand modèle de langage (LLM) comme Gemini sans RAG, il essayerait de répondre uniquement avec ce qu'il a appris pendant son entraînement. Le problème, c'est que Gemini n'a pas été spécifiquement entraîné sur le document de droit du travail. Il pourrait inventer des choses ("halluciner") ou donner des réponses génériques, car il ne "connaît" pas les détails du documents et donc du droit du travail.

**Avec RAG :** Le RAG résout ce problème en ajoutant une étape cruciale : _la recherche d'informations (Retrieval)_ avant la _génération de la réponse (Generation)_.

- **Recherche (Retrieval) :** Quand l'utilisateur poses une question, l'application RAG va d'abord chercher les passages les plus pertinents du document (corpus) qui sont susceptibles de contenir la réponse.
- **Augmentation (Augmented) :** Ces passages pertinents sont ensuite donnés au LLM (Gemini) en même temps que ta question. Le LLM est alors "augmenté" avec des connaissances spécifiques.
- **Génération (Generation) :** Fort de ces informations contextuelles, le LLM peut maintenant générer une réponse précise et basée sur les faits extraits de ton document.